Chameleon/FABRIC setup

In [ ]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease as lease_manager
import chi.server as server_manager
import chi.network as network_manager

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

In [ ]:
# Change to your project (CHI-XXXXXX) 
chi.set("project_name", "CHI-XXXXXX")

chameleon_prefix =  f'{os.environ["USER"]}_fabric_iperf'
chameleon_server_name = chameleon_prefix+'_server'
chameleon_network_name = chameleon_prefix+'_net'
chameleon_subnet_name = chameleon_prefix+'_subnet'
chameleon_router_name = chameleon_prefix+'_router'
chameleon_lease_name = chameleon_prefix+'_lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_server_count=1

chameleon_key_name='<Your_Chameleon_Key>'
chameleon_server_reservation_id = '<Your_Chameleon_Server_Reservation>'

UC reservation

In [ ]:
chi.use_site('CHI@UC')

uc_network_name = chameleon_network_name+'_uc'

# set start/end dates
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)

# create lease reservations
uc_reservations = []
lease_manager.add_node_reservation(uc_reservations, count=1)
lease_manager.add_fip_reservation(uc_reservations, count=1)
reservation_list.append(
        {
            "resource_type": "network",
            "network_name": uc_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# create lease
uc_lease_name = chameleon_lease_name+'_uc'
uc_lease = lease_manager.create_lease(uc_lease_name,
                                     reservations=uc_reservations,
                                     start_date=start_date,
                                     end_date=end_date)

# get lease info
uc_lease_id = lease_manager.get_lease_id(uc_lease_name)
uc_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
uc_node_reservation_id = lease_manager.get_node_reservation(uc_lease_id)

TACC reservation

In [ ]:
chi.use_site('CHI@TACC')

tacc_network_name = chameleon_network_name+'_uc'

# set start/end dates
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)

# create lease reservations
tacc_reservations = []
lease_manager.add_node_reservation(tacc_reservations, count=1)
lease_manager.add_fip_reservation(tacc_reservations, count=1)
reservation_list.append(
        {
            "resource_type": "network",
            "network_name": tacc_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# create lease
tacc_lease_name = chameleon_lease_name+'_uc'
tacc_lease = lease_manager.create_lease(tacc_lease_name,
                                     reservations=tacc_reservations,
                                     start_date=start_date,
                                     end_date=end_date)

# get lease info
tacc_lease_id = lease_manager.get_lease_id(tacc_lease_name)
tacc_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
tacc_node_reservation_id = lease_manager.get_node_reservation(tacc_lease_id)

Check networks

In [ ]:
uc_network_vlan = tacc_network_vlan = None
while uc_network_vlan == None or tacc_network_vlan == None:
    try:
        # uc check
        chi.use_site('CHI@UC')
        
        #Get the network
        uc_network = network_manager.get_network(uc_network_name)

        #Get the network ID
        uc_network_id = uc_network['id']
        # print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        uc_network_vlan = uc_network['provider:segmentation_id']
        # print(f'network_vlan: {network_vlan}')
        
        # tacc check
        chi.use_site('CHI@TACC')
        
        #Get the network
        tacc_network = network_manager.get_network(tacc_network_name)

        #Get the network ID
        tacc_network_id = tacc_network['id']
        # print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        tacc_network_vlan = tacc_network['provider:segmentation_id']
        # print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Networks are not ready. Trying again!')
        time.sleep(10) 